## Exploring Automated Data Viz with Plotly and Slack in Python 📊

The world of journalism is constantly evolving, with new technologies emerging that promise to revolutionize the way news is created, shared, and consumed. In recent years, the rise of automated news and bots has sparked both excitement and apprehension in the industry. Some see these tools as a powerful way to streamline news production, improve accuracy, and reach wider audiences. Others worry that they could lead to job losses and a decline in the quality of journalism. Regardless of where you stand on this issue, there's no denying that the potential of these tools is vast and could pave the way for a new era of journalism.

In this blog post, I'll share my experience with attempting to use an automated Slack bot to send data visualizations as messages. For this project, the two main libraries I used were Plotly and the Slack developer kit for Python. I embarked on this project with the hopes of extending the functionality of my EnviroComplaints bot, which monitors new complaints submitted to the Maryland Department of the Environment. But, as with any coding project, my experience was filled with twists and turns. I'll share more on the process, challenges, and lessons I learned throughput. While I, unfortunately, haven’t fully figured out how to incorporate this into my bot, learning more about these libraries and how to use them was incredibly helpful.

Before we dive in, I wanted to quickly introduce the Plotly and Slack libraries for anyone who hasn’t encountered them before.

[Plotly](https://plotly.com/python/getting-started/) is used for creating interactive graphs and charts. It's a powerful and flexible library that allows users to create a wide variety of visualizations, including scatter plots, line charts, bar graphs, and more. One of the key benefits of Plotly is its interactivity, which allows users to hover over data points and zoom in on specific regions of a graph.

```python
$ pip install plotly
```

The [Slack developer kit for Python](https://python-slackclient.readthedocs.io/en/latest/) allows developers to create custom integrations within the Slack platform. Developers can build custom bots, automate tasks, and integrate Slack with other tools and services.

```python
$ pip install slackclient
```

Now that we've gotten that out of the way, we can start by using Plotly to make a basic horizontal bar chart! Before you run any code in your own environment, make sure you have the latest copy of the complaints_data.csv file, which holds the data we'll be using.

The code block below produces a very basic bar chart, but, hey, it's a start! 

In [2]:
import pandas as pd
import plotly.graph_objs as go

# complaints by county
complaints = pd.read_csv('complaints_data.csv')
# group by 'county' column, count number of complaints per county
complaints_by_county = complaints.groupby(['county']).size().reset_index(name='count')

# find top 10 counties
top_10_counties = complaints_by_county.nlargest(10, 'count')

# horizontal bar chart
fig = go.Figure(go.Bar(
    x=top_10_counties['count'],
    y=top_10_counties['county'],
    orientation='h'
))

fig.write_image("basic_bar.png")

![basic_bar](basic_bar.png)

Next, we'll customize the chart a bit further by changing the color of the bars and adding a title, labels, and even a source tag. How exciting! 

In [3]:
# complaints by county
complaints = pd.read_csv('complaints_data.csv')
# group by 'county' column, count number of complaints per county
complaints_by_county = complaints.groupby(['county']).size().reset_index(name='count')

# find top 10 counties
top_10_counties = complaints_by_county.nlargest(10, 'count')

# horizontal bar chart
fig = go.Figure(go.Bar(
    x=top_10_counties['count'],
    y=top_10_counties['county'],
    orientation='h', 
    text=top_10_counties['count'], # this adds data labels
    textposition='auto', # this adjusts the positioning of the data labels
    marker=dict(color='#87ae73') # this changes the color of the bars to sage green :)
))

# here, we set the chart's title and x-axis label
fig.update_layout(
    title='Top 10 Maryland Counties with the Most Environmental Complaints',
    xaxis_title='Number of Complaints',
    plot_bgcolor='rgba(0,0,0,0)', # this removes that pesky default background
    font=dict(color='black') # this changes the font color to black
)

# here, we're adding a source tag
fig.add_annotation(
    text='Source: Maryland Department of the Environment',
    font=dict(
        family='Arial',
        size=12,
        color='#5A5A5A'
    ),
    # the code below adjusts positioning
    showarrow=False,
    xref='paper',
    yref='paper',
    x=0,
    y=-.22,
    xanchor='left',
    yanchor='bottom'
)

fig.write_image("top_10_counties.png")

![better_bar](top_10_counties.png)

That's much better! Although this is a very simple visualization, it hints to a potentially very interesting story. Why are so many environmental complaints issued in Anne Arundel County? Answering that question requires additional reporting, but I hope you’re starting to see why this kind of functionality is so useful. 

An automated bot that produces data visualizations can be a powerful tool for newsrooms, offering a range of benefits including improved efficiency and the ability to identify patterns and trends in data. Here are a few ways that an automated bot can be used in a newsroom:

* Improved efficiency: Data visualization can be a time-consuming process, especially when dealing with large datasets or complex visualizations. An automated bot can streamline this process by automatically generating visualizations based on predefined parameters, freeing up journalists to focus on other tasks.

* Pattern identification: Data visualization is a powerful tool for identifying patterns and trends in data that might not be immediately apparent through other means. An automated bot can quickly generate a wide range of visualizations, allowing journalists to identify patterns and trends that might otherwise go unnoticed.

* Customizability: An automated bot can be customized to generate specific types of visualizations based on the needs of the newsroom. For example, it could be programmed to automatically generate maps or charts based on data from a specific source or to create visualizations that are optimized for social media sharing.

Well, now that we have a decent looking bar chart, we can start looking at how to merge this with a Slack bot. Full disclosure, this is where I struggled a lot. First, here's my working code for my bot, EnviroComplaints. Everyday, at 9:30 a.m., my bot retrieves the state's latest environmental complaints through the state data portal's API. It then takes that data and identfies the newest ones, and sends a Slack message with details about each complaint.

In [ ]:
import requests
import json
import time
import os
import pandas as pd
from slack import WebClient
from slack.errors import SlackApiError
from datetime import datetime

def check_complaints():
    # setup
    slack_token = os.environ.get('SLACK_API_TOKEN')
    client = WebClient(token=slack_token)

    # api endpoint
    COMPLAINTS_API_ENDPOINT = "https://opendata.maryland.gov/resource/cnkn-n3pr.json"

    try:
        with open('last_checked_date.txt', 'r') as f:
            last_checked_date = f.read().strip()
    except FileNotFoundError:
        with open('last_checked_date.txt', 'w') as f:
            last_checked_date = '2023-03-30'  # default date if file doesn't exist
            f.write(last_checked_date)

    # store api response, check for new complaints
    response = requests.get(COMPLAINTS_API_ENDPOINT)
    complaints_data = json.loads(response.content)

    # save as pandas df, export as csv
    df = pd.DataFrame(complaints_data)
    df.to_csv("complaints_data.csv", index=False)

    new_complaints = []
    for complaint in complaints_data:
        complaint_date = datetime.strptime(complaint['recieved_date'], '%Y-%m-%d')
        last_checked_date_obj = datetime.strptime(last_checked_date, '%Y-%m-%d')
        if complaint_date > last_checked_date_obj:
            new_complaints.append(complaint)

    # slack message
    if new_complaints:
        message = "New complaints :herb:\n\n"
        for complaint in new_complaints:
            if 'compliant' in complaint:
                message += f"  *ID:* {complaint['compliant']}\n"
            if 'recieved_date' in complaint:
                message += f"  *Submitted to MDE:* {complaint['recieved_date']}\n"
            if 'compliant_type' in complaint:
                message += f"  *Complaint Type:* {complaint['compliant_type']}\n"
            if 'incident_date' in complaint:
                message += f"  *Incident Date:* {complaint['incident_date']}\n"
            if 'county' in complaint:
                message += f"  *County:* {complaint['county']}\n"
            if 'incident_status_desc' in complaint:
                message += f"  *Status:* {complaint['incident_status_desc']}\n"
            message += "\n"

        try:
            response = client.chat_postMessage(
                channel="slack-bots",
                text=message
            )
        except SlackApiError as e:
            print(f"Error sending message: {e}")
        else:
            print("Message sent to Slack channel!")

        # update last_checked_date to the received_date of the latest complaint in new_complaints
        last_checked_date = new_complaints[-1]['recieved_date']
        with open('last_checked_date.txt', 'w') as f:
            f.write(last_checked_date)
    else:
        try:
            response = client.chat_postMessage(
                channel="slack-bots",
                text="No new complaints found :herb:"
            )
        except SlackApiError as e:
            print(f"Error sending message: {e}")
        else:
            print("Message sent to Slack channel!")

if __name__ == "__main__":
    check_complaints()

Based on my research and consulting ChatGPT, you could either send an image as an attachment or as a separate message. Here's my take at what that might look like. Again, this didn’t work for me, so I can't guarantee this will work for you. It's a start, but some additional debugging is definitely required! 😬

In [ ]:
# read and turn image into bytes
with open('top_10_counties.png', 'rb') as f:
    img = f.read()

# slack message
try:
    response = client.files_upload(
        channels="slack-bots",
        initial_comment="Top 10 Maryland Counties with the Most Environmental Complaints",
        file=img,
        filename="top_10_counties.png",
        title="Top 10 Maryland Counties with the Most Environmental Complaints"
    )
except SlackApiError as e:
    print(f"Error sending message: {e}")
else:
    print("Message sent to Slack channel!")

### Lessons Learned

* Automation requires patience: Automating a task can be a great way to streamline a process, but it can also be frustrating and time-consuming. The process of building an automated bot requires patience and persistence, as well as the ability to troubleshoot and problem-solve when things don't go as planned.
* Data visualization can reveal unexpected insights: One of the most powerful aspects of data visualization is its ability to reveal patterns and insights that might not be immediately apparent from raw data. By automating the data visualization process, it's possible to quickly generate a wide range of visualizations that can help to uncover unexpected insights and trends.
* The importance of flexibility and adaptability: While working on this project, I realized that flexibility and adaptability are key qualities that every data journalist should have. There were several instances where I had to change my approach and pivot my angle to achieve my desired outcome. For example, I initially planned to use Plotly to produce a choropleth map of Maryland, but when I encountered issues with using shapefiles and GeoJSON, I turned to the basics. Being able to adapt and adjust your approach based on the challenges you face is a crucial skill that can make the difference between success and failure.

While I wasn't able to achieve my initial goal, I found the experience to be a valuable learning opportunity. Working with Plotly and the Slack developer kit for Python allowed me to push myself further and think creatively about how I could improve my bot. The challenges I encountered along the way taught me important lessons about the importance of flexibility and adaptability. While it can be frustrating when projects don't go as planned, it's important to remember that every failure can be an opportunity for growth and improvement.

### Resources

Slack Documentation

* [Creating rich message layouts](https://api.slack.com/messaging/composing/layouts)
* [files.upload](https://api.slack.com/methods/files.upload)

Other Blog Posts & Tutorials
* [Send messages, images and Pandas Data Frames to Slack with Python](https://medium.com/@alexandre.tkint/send-messages-images-and-pandas-data-frames-to-slack-with-python-2dd9929c356e) by Alexandre t'Kint
* [Slack: Send Attachments Using Python WebClient](https://pandeynandancse.medium.com/slack-send-attachments-using-python-webclient-baa39974ceea) by Nandan Pandey
* [How to send dynamic charts with a Slack bot](https://quickchart.io/documentation/send-charts-with-slack-bot/)